In [50]:
import pandas as pd
import os
import pyodbc
import urllib
import datetime
# Import important sqlalchemy classes
#
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True
conn_string_db = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=ACCIDENTS;Trusted_Connection=yes;"
conn_string_db = urllib.parse.quote_plus(conn_string_db)
conn_string_db = "mssql+pyodbc:///?odbc_connect=%s" % conn_string_db
conn_string_dw = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=OLAP;Trusted_Connection=yes;"
conn_string_dw = urllib.parse.quote_plus(conn_string_dw)
conn_string_dw = "mssql+pyodbc:///?odbc_connect=%s" % conn_string_dw

In [10]:
def load_drivers(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct Age_Band_of_Driver, Age_of_Vehicle, Driver_Home_Area_Type, Driver_IMD_Decile, Journey_Purpose_of_Driver, Sex_of_Driver '
    sqlQuery += f'from Vehicle'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class DimDriver(Base):
        __tablename__ = 'Driver'
#   The ORM DimCustomers object bound to the database table DimCustomers
        Driver_id = Column(Integer, primary_key=True, autoincrement=True) 
        Age_Band_of_Driver = Column(String)
        Age_of_Vehicle  = Column(Integer)
        Driver_Home_Area_Type  = Column(String)
        Driver_IMD_Decile  = Column(Integer)
        Journey_Purpose_of_Driver  = Column(String)
        Sex_of_Driver  = Column(String)
#
# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
            # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
            new_record = DimDriver(Age_Band_of_Driver=row['Age_Band_of_Driver'], Age_of_Vehicle=row['Age_of_Vehicle'], 
                                Driver_Home_Area_Type=row['Driver_Home_Area_Type'],
                                  Driver_IMD_Decile=row['Driver_IMD_Decile'], Journey_Purpose_of_Driver=row['Journey_Purpose_of_Driver'], 
                                  Sex_of_Driver=row['Sex_of_Driver'])
            session.add(new_record)
            counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Driver = ' + str(counter)
        f.write(out_str)
load_drivers(conn_string_db,conn_string_dw)

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\1636422750.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [14]:
def load_automobile(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct Accident_Index,make,model,Engine_Capacity_CC,Propulsion_Code,Vehicle_Type,Was_Vehicle_Left_Hand_Drive '
    sqlQuery += f'from Vehicle'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class DimAutomobile(Base):
        __tablename__ = 'Automobile_details'
#   The ORM DimCustomers object bound to the database table DimCustomers
        Automobile_id = Column(Integer, primary_key=True, autoincrement=True) 
        Accident_Index = Column(String)
        make = Column(String)
        model = Column(String)
        Engine_Capacity_CC = Column(Integer)
        Propulsion_Code = Column(String)
        Vehicle_Type = Column(String)
        Was_Vehicle_Left_Hand_Drive = Column(String)
        
        
#
# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
            # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
            new_record = DimAutomobile(Accident_Index=row['Accident_Index'], make=row['make'], model=row['model'],
                                  Engine_Capacity_CC=row['Engine_Capacity_CC'], Propulsion_Code=row['Propulsion_Code'], 
                                  Vehicle_Type=row['Vehicle_Type'],Was_Vehicle_Left_Hand_Drive=row['Was_Vehicle_Left_Hand_Drive'])
            session.add(new_record)
            counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Automobile_details table = ' + str(counter)
        f.write(out_str)
load_automobile(conn_string_db,conn_string_dw)

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\125543513.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
def load_Accident_details(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct Accident_Index,make,model,Engine_Capacity_CC,Propulsion_Code,Vehicle_Type,Was_Vehicle_Left_Hand_Drive '
    sqlQuery += f'from Vehicle'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class Accident_details(Base):
        __tablename__ = 'Accident_details'
#   The ORM DimCustomers object bound to the database table DimCustomers
        Accident_details_id = Column(Integer, primary_key=True, autoincrement=True) 
        Hit_Object_in_Carriageway = Column(String)
        Hit_Object_off_Carriageway = Column(String)
        Junction_Location = Column(String)
        Skidding_and_Overturning = Column(String)
        Towing_and_Articulation = Column(String)
        Vehicle_Leaving_Carriageway = Column(String)
        Vehicle_Location_Restricted_Lane = Column(Integer)
        Vehicle_Manoeuvre = Column(String)
        X1st_Point_of_Impact = Column(String)
        
        
#
# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
            # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
            new_record = Accident_details(Accident_details_id=row['Accident_details_id'], Hit_Object_in_Carriageway=row['Hit_Object_in_Carriageway'],
                                          Hit_Object_off_Carriageway=row['Hit_Object_off_Carriageway'],
                                  Junction_Location=row['Junction_Location'], Skidding_and_Overturning=row['Skidding_and_Overturning'], 
                                  Towing_and_Articulation=row['Towing_and_Articulation'],Vehicle_Leaving_Carriageway=row['Vehicle_Leaving_Carriageway'],
                                Vehicle_Location_Restricted_Lane=row['Vehicle_Location_Restricted_Lane'],Vehicle_Manoeuvre=row['Vehicle_Manoeuvre'],     )
            session.add(new_record)
            counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Accident_details table = ' + str(counter)
        f.write(out_str)
load_Accident_details(conn_string_db,conn_string_dw)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import datetime
import numpy as np

def load_ft_vehicles_involved(conStrdb, conStrdw):
    # Engines for the databases
    engine_db = create_engine(conStrdb)  # Source database (a)
    engine_dw = create_engine(conStrdw)  # Destination database (b)

    # Query data from database 'a'
    sql_query_a = """
    SELECT a.Accident_Index, Vehicle_Reference, Year
    FROM Vehicle v JOIN Accident_Location a ON v.Accident_Index = a.Accident_Index
    """
    df_a = pd.read_sql_query(sql_query_a, engine_db)
    sql_query_b = "SELECT * FROM Accident_details"
    df_b = pd.read_sql_query(sql_query_b, engine_dw)

    sql_query_c = "SELECT * FROM Automobile_details"
    df_c = pd.read_sql_query(sql_query_c, engine_dw)

    sql_query_d = "SELECT * FROM Driver"
    df_d = pd.read_sql_query(sql_query_d, engine_dw)

    sql_query_e = "SELECT v.*,a.Junction_Location,a.Year FROM Vehicle v left join Accident_Location a on v.Accident_Index=a.Accident_Index "
    df_e = pd.read_sql_query(sql_query_e, engine_db)

    df_a = df_a.merge(df_e, how='left', on=['Accident_Index', 'Vehicle_Reference','Year'])

    df_a = df_a.merge(df_b, how='left', on=df_b.columns.to_list().remove('Accident_details_id'))

    df_a = df_a.merge(df_c, how='left', on=df_c.columns.to_list().remove('Automobile_id'))

    df_a = df_a.merge(df_d, how='left', on=df_d.columns.to_list().remove('Driver_id'))

    df_a=df_a[['Accident_Index','Vehicle_Reference','Year','Driver_id','Automobile_id','Accident_details_id']]

    df_a.replace({np.nan: None}, inplace=True)

    # Define ORM for table 'FT_Vehicles_Involved' in database 'b'
    Base = declarative_base()
    class FT_Vehicles_Involved(Base):
        __tablename__ = 'FT_Vehicles_Involved'
        Accident_Index = Column(String, primary_key=True)
        Vehicle_Reference = Column(String, primary_key=True)
        Year = Column(Integer)
        Driver_id = Column(Integer)
        Automobile_id = Column(Integer)
        Accident_details_id = Column(Integer)

    # Insert data into table 'FT_Vehicles_Involved' in database 'b'
    Base.metadata.create_all(engine_dw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in df_a.iterrows():
        new_record = FT_Vehicles_Involved(Accident_Index=row['Accident_Index'], Vehicle_Reference=row['Vehicle_Reference'], Year=row['Year'],
                                         Driver_id=row['Driver_id'], Automobile_id=row['Automobile_id'], Accident_details_id=row['Accident_details_id'])
        session.add(new_record)
        counter += 1
    session.commit()
   
    session.close()

    # Log the update
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        f.write("All the foreign keys have been inserted into the Fact table")
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into FT_Vehicles_Involved table = ' + str(counter)
        f.write(out_str)

# Example usage
load_ft_vehicles_involved(conn_string_db, conn_string_dw)


In [44]:
def load_road(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct First_Road_Class,First_Road_Number,Second_Road_Class,Second_Road_Number,Light_Conditions,Road_Surface_Conditions,Road_Type,Special_Conditions_At_Site,Speed_Limit  '
    sqlQuery += f'from Accidents'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class DimRoad(Base):
        __tablename__ = 'Road'
        Road_id = Column(Integer, primary_key=True, autoincrement=True)
        First_Road_Class = Column(String)
        First_Road_Number = Column(Integer)
        Second_Road_Class = Column(String)
        Second_Road_Number = Column(Integer)
        Light_Conditions = Column(String)
        Road_Surface_Conditions = Column(String)
        Road_Type = Column(String)
        Special_Conditions_At_Site = Column(String)
        Speed_Limit = Column(Integer)
#   The ORM DimCustomers object bound to the database table DimCustomers


# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
            # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
            new_record = DimRoad(First_Road_Class=row['First_Road_Class'], First_Road_Number=row['First_Road_Number'],
                                  Second_Road_Class=row['Second_Road_Class'], Second_Road_Number=row['Second_Road_Number'], 
                                  Light_Conditions=row['Light_Conditions'],Road_Surface_Conditions=row['Road_Surface_Conditions'],
                                  Road_Type=row['Road_Type'],Special_Conditions_At_Site=row['Special_Conditions_At_Site'],
                                    Speed_Limit=row['Speed_Limit'])
                                      
            session.add(new_record)
            counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Road table = ' + str(counter)
        f.write(out_str)
load_road(conn_string_db,conn_string_dw)

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\325348239.py:15: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [45]:
def load_Acc_Incidents(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct Accident_Severity,Carriageway_Hazards,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Control,Junction_Detail  '
    sqlQuery += f'from Accidents'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class DimAcc_Incidents(Base):
        __tablename__ = 'Acc_Incidents'
        Acc_Incidents_id = Column(Integer, primary_key=True, autoincrement=True)
        Accident_Severity = Column(String)
        Carriageway_Hazards = Column(String)
        Did_Police_Officer_Attend_Scene_of_Accident = Column(Integer)
        Junction_Control = Column(String)
        Junction_Detail = Column(String)
#   The ORM DimCustomers object bound to the database table DimCustomers


# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
            # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
            new_record = DimAcc_Incidents(Accident_Severity=row['Accident_Severity'],
                                  Carriageway_Hazards=row['Carriageway_Hazards'], Did_Police_Officer_Attend_Scene_of_Accident=row['Did_Police_Officer_Attend_Scene_of_Accident'], 
                                  Junction_Control=row['Junction_Control'],Junction_Detail=row['Junction_Detail'])
                                      
            session.add(new_record)
            counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Acc_Incidents table = ' + str(counter)
        f.write(out_str)
load_Acc_Incidents(conn_string_db,conn_string_dw)

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\1632606768.py:15: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [46]:
from sqlalchemy import create_engine, Column, Integer, String , Float
def load_Location(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct l.Latitude, l.Longitude, Location_Easting_OSGR ,Location_Northing_OSGR ,Police_Force ,LSOA_Of_Accident_Location ,Urban_Or_Rural_Area,InScotland ,Local_District_Authority ,Local_Highway_Authority ,Pedestrian_Crossing_Human_Control ,Pedestrian_Crossing_Physical_Facilities  '
    sqlQuery += f'from Accidents a join Location l on a.Latitude=l.Latitude and a.Longitude=l.Longitude'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class DimLocation(Base):
        __tablename__ = 'Location'

        Location_id = Column(Integer, primary_key=True, autoincrement=True)
        Latitude = Column(Float)
        Longitude= Column(Float) 
        Location_Easting_OSGR= Column(String)
        Location_Northing_OSGR= Column(String)
        Police_Force= Column(String)
        LSOA_Of_Accident_Location= Column(String)
        Urban_Or_Rural_Area= Column(String)
        InScotland= Column(String)
        Local_District_Authority= Column(String)
        Local_Highway_Authority= Column(String)
        Pedestrian_Crossing_Human_Control= Column(Integer)
        Pedestrian_Crossing_Physical_Facilities= Column(Integer)
#   The ORM DimCustomers object bound to the database table DimCustomers


# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
        # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
        new_record = DimLocation(Latitude=row['Latitude'],
                                Longitude=row['Longitude'], 
                                    Location_Easting_OSGR=row['Location_Easting_OSGR'], 
                                    Location_Northing_OSGR=row['Location_Northing_OSGR'], 
                                    Police_Force=row['Police_Force'], 
                                    LSOA_Of_Accident_Location=row['LSOA_Of_Accident_Location'], 
                                    Urban_Or_Rural_Area=row['Urban_Or_Rural_Area'], 
                                    InScotland=row['InScotland'], 
                                    Local_District_Authority=row['Local_District_Authority'], 
                                    Local_Highway_Authority=row['Local_Highway_Authority'], 
                                    Pedestrian_Crossing_Human_Control=row['Pedestrian_Crossing_Human_Control'], 
                                    Pedestrian_Crossing_Physical_Facilities=row['Pedestrian_Crossing_Physical_Facilities']
                                    )
                                      
        session.add(new_record)
        counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Location table = ' + str(counter)
        f.write(out_str)
load_Location(conn_string_db,conn_string_dw)

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\1518115383.py:16: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [47]:
from sqlalchemy import create_engine, Column, Integer, String , Float , Time , Date
def load_date(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    sqlQuery = f'select distinct  d.Date,Day_of_Week,Time,Year  '
    sqlQuery += f'from Dates d join Accidents a on d.Date=a.Date'
    dFrame = pd.read_sql_query(sqlQuery, engine_db)

    import numpy as np

# Assuming 'dFrame' is your DataFrame
    dFrame.replace({np.nan: None}, inplace=True)

# Now continue with your data insertion as before

#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class DimDate(Base):
        __tablename__ = 'Date'

        Date_id = Column(Integer, primary_key=True, autoincrement=True)
        Date = Column(Date)
        Day_of_Week= Column(String) 
        Time= Column(Time)
        Year= Column(Integer)


        
#   The ORM DimCustomers object bound to the database table DimCustomers


# Create an ORM session connected to the dimensional database MyStoreDWETL using the connection-string conStrdw
    engine_dw = create_engine(conStrdw)
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0
    for _, row in dFrame.iterrows():
        
        # If primary key does not exist customer_id from dFrame (customers table) is absent in DimCustomers
            #, create a new object and add it to the session
        new_record = DimDate(Date=row['Date'],
                                Day_of_Week=row['Day_of_Week'], 
                                    Time=row['Time'], 
                                    Year=row['Year']
                                    )
                                      
        session.add(new_record)
        counter += 1

    # Commit the session to persist the changes to the database
    session.commit()
    # Close the session
    session.close()
###    
    with open('task2.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        f.write('\n')
        f.write('------------------------------------------------------------------------------------------------')
        f.write('\n')
        f.write('\n')
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Date table = ' + str(counter)
        f.write(out_str)
load_date(conn_string_db,conn_string_dw)

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\3039684542.py:16: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [63]:
import pandas as pd
import os
import pyodbc
import urllib
import datetime
import numpy as np
# Import important sqlalchemy classes
#
from sqlalchemy import create_engine, Column, Integer, String, Date, Float , Time
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True
#
 
conn_string_db = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=ACCIDENTS;Trusted_Connection=yes;"
conn_string_db = urllib.parse.quote_plus(conn_string_db)
conn_string_db = "mssql+pyodbc:///?odbc_connect=%s" % conn_string_db
conn_string_dw = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=OLAP;Trusted_Connection=yes;"
conn_string_dw = urllib.parse.quote_plus(conn_string_dw)
conn_string_dw = "mssql+pyodbc:///?odbc_connect=%s" % conn_string_dw
 
 
def create_Date_Key_df(inFrame, column_name):
    in_Year = pd.to_datetime(inFrame[column_name]).dt.year
    in_Month = pd.to_datetime(inFrame[column_name]).dt.strftime('%m')
    in_DayNumber = pd.to_datetime(inFrame[column_name]).dt.strftime('%d')
    out_Date_Key = (in_Year.astype(str) + in_Month.astype(str) + in_DayNumber.astype(str)).astype(int)
    return out_Date_Key
#####################################################################################
def addNew_Date(chkDate: datetime, conStrdw):
    engine_dw = create_engine(conStrdw)
    sqlQuery = f'SELECT * FROM DimDate'
    dFrame = pd.read_sql_query(sqlQuery, engine_dw)
#   print(dFrame)
#
    Base = declarative_base()
    class DimDate(Base):
        __tablename__ = 'DimDate'
        Date_Key = Column(Integer, primary_key=True, autoincrement=False)
        Date = Column(Date)
        Year  = Column(Integer)
        Month  = Column(Integer)
        MonthName = Column(String)
        Week  = Column(Integer)
        WeekDay  = Column(Integer)
        WeekDayName = Column(String)
        DayNumber = Column(Integer)

#
    chk_Date = chkDate.strftime("%Y-%m-%d")
    chk_Year = int(chkDate.year)
    chk_Month = chkDate.strftime("%m")
    chk_MonthName =chkDate.strftime('%B')
    chk_Week = int(chkDate.strftime('%U')) + 1
    chk_WeekDay = chkDate.weekday()+2
    chk_WeekDayName = chkDate.strftime('%A')
    chk_DayNumber = chkDate.strftime("%d")
    chk_Date_Key = int (str(chk_Year) + str(chk_Month) + str(chk_DayNumber))
#   print(chk_Date_Key, chk_Date, chk_Year, chk_Month, chk_MonthName, chk_Week, chk_WeekDay, chk_WeekDayName, chk_DayNumber)    
#
# Create a session
    Session = sessionmaker(bind=engine_dw)
    session = Session()
#
    if not session.query(DimDate).filter_by(Date_Key=chk_Date_Key).first():
        # If primary key does not exist, create a new object and add it to the session
        new_record = DimDate(Date_Key=chk_Date_Key, Date = chk_Date, Year = chk_Year, Month = chk_Month, MonthName = chk_MonthName,
                              Week=chk_Week, WeekDay=chk_WeekDay, WeekDayName=chk_WeekDayName, DayNumber = chk_DayNumber)
        session.add(new_record)
        # Commit the session to persist the changes to the database
        session.commit()
        print("Date rows successfully inserted into SQL Server table.")
        # Close the session
        session.close()
    else:
          print(f"Skipping duplicate primary key:", chk_Date_Key)
#####################################################################################
def pipe_Accident(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    engine_dw = create_engine(conStrdw)

        # Query data from database 'a'
    sql_query_a = """
    select Accident_Index,Number_of_Casualities,Number_of_Vehicles
    from Accidents
    """
    df_a = pd.read_sql_query(sql_query_a, engine_db)
    sql_query_e = "SELECT a.*,d.Day_Of_Week,d.Year,l.Location_Easting_OSGR,l.Location_Northing_OSGR,l.Police_Force,l.LSOA_Of_Accident_Location,l.Urban_Or_Rural_Area,l.InScotland FROM Accidents a left join Dates d on a.Date=d.Date left join Location l on l.Latitude=a.Latitude and l.Longitude=a.Longitude  "
    df_e = pd.read_sql_query(sql_query_e, engine_db)

    sql_query_b = "SELECT * FROM Date"
    df_b = pd.read_sql_query(sql_query_b, engine_dw)
    sql_query_c = "SELECT * FROM Acc_Incidents"
    df_c = pd.read_sql_query(sql_query_c, engine_dw)
    sql_query_d = "SELECT * FROM Location"
    df_d = pd.read_sql_query(sql_query_d, engine_dw)
    sql_query_f = "SELECT * FROM Road"
    df_f = pd.read_sql_query(sql_query_f, engine_dw)
    sql_query_g = "SELECT * FROM Automobile_details"
    df_g = pd.read_sql_query(sql_query_g, engine_dw)
    df_a= df_a.merge(df_e, how='left', on=['Accident_Index', 'Number_of_Casualities','Number_of_Vehicles'])
    df_a = df_a.merge(df_b, how='left', on=df_b.columns.to_list().remove('Date_id'))
    df_a=df_a.merge(df_c, how='left', on=df_c.columns.to_list().remove('Acc_Incidents_id'))
    df_a=df_a.merge(df_d, how='left', on=df_d.columns.to_list().remove('Location_id'))
    df_a=df_a.merge(df_f, how='left', on=df_f.columns.to_list().remove('Road_id'))
    df_a=df_a.merge(df_g, how='left', on=df_g.columns.to_list().remove('Automobile_id'))
    df_a=df_a[['Accident_Index', 'Number_of_Casualities', 'Number_of_Vehicles','Acc_Incidents_id','Date','Date_id','Location_id', 'Road_id', 'Automobile_id']]
    df_a.replace({np.nan: None}, inplace=True)
 
#
#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class FACT_Accident(Base):
        __tablename__ = 'FT_Accident'
        FT_Accident_ID = Column(Integer, primary_key=True, autoincrement=True)
        Accident_Index = Column(String)
        Number_of_Casualities = Column(Integer)
        Number_of_Vehicles = Column(Integer)
        Location_id= Column(Integer)
        Road_id = Column(Integer)        
        Acc_Incidents_id = Column(Integer)
        Automobile_id = Column(Integer)
        Accident_Date_key = Column(Integer)
        Accident_date = Column(Date)
        Date_id= Column(Integer)

 
        
    class DimDate(Base):
        __tablename__ = 'DimDate'
        Date_Key = Column(Integer, primary_key=True, autoincrement=False)
        Date = Column(Date)
        Year  = Column(Integer)
        Month  = Column(Integer)
        MonthName = Column(String)
        Week  = Column(Integer)
        WeekDay  = Column(Integer)
        WeekDayName = Column(String)
        DayNumber = Column(Integer)
    df_a['Accident_Date_key'] = create_Date_Key_df(df_a, 'Date')
# Create a session
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0

    for _, row in df_a.iterrows():
        if not session.query(DimDate).filter_by(Date_Key=row['Accident_Date_key']).first():
            # If Order date does not exist in DimDate, add it to DimDate
            addNew_Date(row['Date'],conn_string_dw)         
    for _, row in df_a.iterrows():
        if not session.query(FACT_Accident).filter_by(Accident_Index=row['Accident_Index']).first():
        #print(row['order_id'])
            new_record = FACT_Accident(Accident_Index=row['Accident_Index'], Number_of_Casualities=row['Number_of_Casualities'],
                                   Number_of_Vehicles=row['Number_of_Vehicles'], Acc_Incidents_id=row['Acc_Incidents_id'],
                                   Date=row['Date'], Date_id=row['Date_id'], Location_id=row['Location_id'], 
                                   Road_id=row['Road_id'],Automobile_id=row['Automobile_id'],Accident_Date_key=row['Accident_Date_key'])
            session.add(new_record)
            counter+=1
            session.commit()
    session.commit()
    print("DataFrame rows successfully inserted into SQL Server table.")
    # Close the session
    session.close()

 
    
    with open('pipe_Accident.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Fact_Accident = ' + str(counter)
        f.write(out_str)
#####################################################################################
pipe_Accident(conn_string_db, conn_string_dw)
#####################################################################################

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\3475588948.py:113: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
C:\Users\phani\AppData\Local\Temp\ipykernel_22600\3475588948.py:37: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows successfully inserted into SQL Server table.
Date rows 

TypeError: 'Date' is an invalid keyword argument for FACT_Accident

In [65]:
import pandas as pd
import os
import pyodbc
import urllib
import datetime
import numpy as np
# Import important sqlalchemy classes
#
from sqlalchemy import create_engine, Column, Integer, String, Date, Float , Time
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True
#
 
conn_string_db = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=ACCIDENTS;Trusted_Connection=yes;"
conn_string_db = urllib.parse.quote_plus(conn_string_db)
conn_string_db = "mssql+pyodbc:///?odbc_connect=%s" % conn_string_db
conn_string_dw = "Driver={ODBC Driver 17 for SQL Server};Server=MY_GENIE\SQLEXPRESS;Database=OLAP;Trusted_Connection=yes;"
conn_string_dw = urllib.parse.quote_plus(conn_string_dw)
conn_string_dw = "mssql+pyodbc:///?odbc_connect=%s" % conn_string_dw
 
 
def create_Date_Key_df(inFrame, column_name):
    in_Year = pd.to_datetime(inFrame[column_name]).dt.year
    in_Month = pd.to_datetime(inFrame[column_name]).dt.strftime('%m')
    in_DayNumber = pd.to_datetime(inFrame[column_name]).dt.strftime('%d')
    out_Date_Key = (in_Year.astype(str) + in_Month.astype(str) + in_DayNumber.astype(str)).astype(int)
    return out_Date_Key
#####################################################################################



def pipe_Accident(conStrdb, conStrdw):
    engine_db = create_engine(conStrdb)
    engine_dw = create_engine(conStrdw)

        # Query data from database 'a'
    sql_query_a = """
    select Accident_Index,Number_of_Casualities,Number_of_Vehicles
    from Accidents
    """
    df_a = pd.read_sql_query(sql_query_a, engine_db)
    sql_query_e = "SELECT a.*,d.Day_Of_Week,d.Year,l.Location_Easting_OSGR,l.Location_Northing_OSGR,l.Police_Force,l.LSOA_Of_Accident_Location,l.Urban_Or_Rural_Area,l.InScotland FROM Accidents a left join Dates d on a.Date=d.Date left join Location l on l.Latitude=a.Latitude and l.Longitude=a.Longitude  "
    df_e = pd.read_sql_query(sql_query_e, engine_db)

    sql_query_b = "SELECT * FROM Date"
    df_b = pd.read_sql_query(sql_query_b, engine_dw)
    sql_query_c = "SELECT * FROM Acc_Incidents"
    df_c = pd.read_sql_query(sql_query_c, engine_dw)
    sql_query_d = "SELECT * FROM Location"
    df_d = pd.read_sql_query(sql_query_d, engine_dw)
    sql_query_f = "SELECT * FROM Road"
    df_f = pd.read_sql_query(sql_query_f, engine_dw)
    sql_query_g = "SELECT * FROM Automobile_details"
    df_g = pd.read_sql_query(sql_query_g, engine_dw)
    df_a= df_a.merge(df_e, how='left', on=['Accident_Index', 'Number_of_Casualities','Number_of_Vehicles'])
    df_a = df_a.merge(df_b, how='left', on=df_b.columns.to_list().remove('Date_id'))
    df_a=df_a.merge(df_c, how='left', on=df_c.columns.to_list().remove('Acc_Incidents_id'))
    df_a=df_a.merge(df_d, how='left', on=df_d.columns.to_list().remove('Location_id'))
    df_a=df_a.merge(df_f, how='left', on=df_f.columns.to_list().remove('Road_id'))
    df_a=df_a.merge(df_g, how='left', on=df_g.columns.to_list().remove('Automobile_id'))
    df_a=df_a[['Accident_Index', 'Number_of_Casualities', 'Number_of_Vehicles','Acc_Incidents_id','Date','Date_id','Location_id', 'Road_id', 'Automobile_id']]
    df_a.replace({np.nan: None}, inplace=True)
 
#
#   This is ORM whereby an object is created corresponding to the database table DimCustomers
    Base = declarative_base()
    class FACT_Accident(Base):
        __tablename__ = 'FT_Accident'
        FT_Accident_ID = Column(Integer, primary_key=True, autoincrement=True)
        Accident_Index = Column(String)
        Number_of_Casualities = Column(Integer)
        Number_of_Vehicles = Column(Integer)
        Location_id= Column(Integer)
        Road_id = Column(Integer)        
        Acc_Incidents_id = Column(Integer)
        Automobile_id = Column(Integer)
        Accident_Date_key = Column(Integer)
        Accident_date = Column(Date)
        Date_id= Column(Integer)


    df_a['Accident_Date_key'] = create_Date_Key_df(df_a, 'Date')
# Create a session
    Session = sessionmaker(bind=engine_dw)
    session = Session()
    counter = 0

      
    for _, row in df_a.iterrows():
        #if not session.query(FACT_Accident).filter_by(Accident_Index=row['Accident_Index']).first():
        #print(row['order_id'])
        new_record = FACT_Accident(Accident_Index=row['Accident_Index'], Number_of_Casualities=row['Number_of_Casualities'],
                                Number_of_Vehicles=row['Number_of_Vehicles'], Acc_Incidents_id=row['Acc_Incidents_id'],
                                Accident_date=row['Date'], Date_id=row['Date_id'], Location_id=row['Location_id'], 
                                Road_id=row['Road_id'],Automobile_id=row['Automobile_id'],Accident_Date_key=row['Accident_Date_key'])
        session.add(new_record)
        counter+=1
        session.commit()
    session.commit()
    print("DataFrame rows successfully inserted into SQL Server table.")
    # Close the session
    session.close()

 
    
    with open('pipe_Accident.txt', 'a') as f:
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        out_str = 'TimeStamp: ' + dt_str + ' --- Number of new  records loaded into Fact_Accident = ' + str(counter)
        f.write(out_str)
#####################################################################################
pipe_Accident(conn_string_db, conn_string_dw)
#####################################################################################

C:\Users\phani\AppData\Local\Temp\ipykernel_22600\1057231263.py:68: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


DataFrame rows successfully inserted into SQL Server table.
